In [1]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import warnings
#suppress warnings
warnings.filterwarnings('ignore')
print("Imports successful")

Imports successful


In [2]:
dataset = pd.read_csv(Path("../regressionAssignment/insurance_pre.csv"))
dataset = pd.get_dummies(dataset,drop_first=True)

independent_vars = ["age","bmi","children","sex_male","smoker_yes"]
dependent_var = "charges"

x = dataset[independent_vars]
y=dataset[dependent_var]

X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=0)
print("created training and test dataset")

created training and test dataset


In [3]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

param_grid = {
    'kernel': ['rbf', 'poly', 'sigmoid', 'linear'],
    'C': [10, 100, 1000, 2000, 3000],
    'gamma': ['auto', 'scale']
}

#refit if true will fit the model for best hyperparameter combination
grid = GridSearchCV(SVR(), param_grid, refit=True, verbose=3, n_jobs=-1)

# fitting the model for grid search
grid.fit(X_train, y_train)


Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV 2/5] END ......C=10, gamma=auto, kernel=rbf;, score=0.013 total time=   0.0s
[CV 5/5] END .....C=10, gamma=auto, kernel=rbf;, score=-0.102 total time=   0.0s
[CV 1/5] END ......C=10, gamma=auto, kernel=rbf;, score=0.004 total time=   0.0s
[CV 4/5] END .....C=10, gamma=auto, kernel=rbf;, score=-0.095 total time=   0.0s
[CV 4/5] END ....C=10, gamma=auto, kernel=poly;, score=-0.025 total time=   0.0s
[CV 5/5] END ....C=10, gamma=auto, kernel=poly;, score=-0.050 total time=   0.0s
[CV 1/5] END .....C=10, gamma=auto, kernel=poly;, score=0.056 total time=   0.1s
[CV 3/5] END .C=10, gamma=auto, kernel=sigmoid;, score=-0.047 total time=   0.1s
[CV 1/5] END ..C=10, gamma=auto, kernel=sigmoid;, score=0.050 total time=   0.1s
[CV 2/5] END ..C=10, gamma=auto, kernel=sigmoid;, score=0.076 total time=   0.1s
[CV 3/5] END .....C=10, gamma=auto, kernel=rbf;, score=-0.104 total time=   0.1s
[CV 2/5] END ...C=10, gamma=auto, kernel=linear

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",SVR()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'C': [10, 100, ...], 'gamma': ['auto', 'scale'], 'kernel': ['rbf', 'poly', ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",None
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 

In [5]:
#print best parameter
print(f"Best parameter={grid.best_params_}")

Best parameter={'C': 3000, 'gamma': 'scale', 'kernel': 'poly'}


In [6]:
#Let us see the results in tabular format
res_dict = grid.cv_results_
res_table = pd.DataFrame.from_dict(res_dict)
res_table


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.033989,0.013342,0.015050,0.003259,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.004055,0.013366,-0.103821,-0.095119,-0.101604,-0.056625,0.053504,35
1,0.031479,0.025293,0.004514,0.001209,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.056274,0.069532,-0.045601,-0.025079,-0.049592,0.001107,0.051309,32
2,0.051736,0.019448,0.005610,0.004452,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.049905,0.075905,-0.046585,-0.041004,-0.046507,-0.001657,0.053391,34
3,0.018257,0.006399,0.002986,0.001754,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.377969,0.479601,0.317872,0.337979,0.324422,0.367569,0.059777,25
4,0.015279,0.003371,0.010834,0.003145,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.004126,0.013244,-0.103775,-0.095165,-0.101602,-0.056634,0.053486,36
5,0.020828,0.005658,0.003339,0.002928,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.054964,0.071297,-0.046513,-0.024157,-0.049652,0.001188,0.051594,31
6,0.028711,0.004637,0.003441,0.000957,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.049644,0.076323,-0.046798,-0.040824,-0.046521,-0.001635,0.053474,33
7,0.015776,0.010329,0.002174,0.000998,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.377969,0.479601,0.317872,0.337979,0.324422,0.367569,0.059777,25
8,0.018629,0.003849,0.009420,0.004491,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.300573,0.339474,0.173708,0.217991,0.183375,0.243024,0.065733,29
9,0.014220,0.005047,0.003785,0.001874,100,auto,poly,"{'C': 100, 'gamma': 'auto', 'kernel': 'poly'}",0.540643,0.575051,0.474839,0.535730,0.424640,0.510181,0.053582,21


In [ ]:
#Let us gather inputs. This does not work in VsCode. 
#It only works in Jupyter notebook
age = int(input("Enter Age:"))
bmi = float(input("Enter BMI:"))
children = int(input("No. of children:"))
sex_male = bool(input("male?:"))
smoker = bool(input("smoker?:"))


In [ ]:
#Let us do prediction
prediction = grid.predict([[age,bmi,children,sex_male,smoker]])
print(f"prediction={prediction}")